In [1]:
import numpy as np
import pandas as pd

In [2]:
!pip install pycocotools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.2/426.2 kB 8.6 MB/s eta 0:00:00:00:01


In [3]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.models.detection import RetinaNet
from torchvision.datasets import CocoDetection
from pycocotools.coco import COCO
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os


In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = CocoDetection(root='/kaggle/input/coco-2017-dataset/coco2017/train2017', annFile='/kaggle/input/coco-2017-dataset/coco2017/annotations/instances_train2017.json', transform=transform)
val_dataset = CocoDetection(root='/kaggle/input/coco-2017-dataset/coco2017/val2017', annFile='/kaggle/input/coco-2017-dataset/coco2017/annotations/instances_val2017.json', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=4, collate_fn=lambda x: tuple(zip(*x)), drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=4, collate_fn=lambda x: tuple(zip(*x)), drop_last=True)

loading annotations into memory...
Done (t=24.65s)
creating index...
index created!
loading annotations into memory...
Done (t=0.98s)
creating index...
index created!


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        targets = torch.tensor(targets)
        BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1 - pt) ** self.gamma * BCE_loss
        
        if self.reduction == 'mean':
            return F_loss.mean()
        elif self.reduction == 'sum':
            return F_loss.sum()
        else:
            return F_loss

focal_loss = FocalLoss()


In [6]:
class Backbone(nn.Module):
    def __init__(self, model_name='resnet50'):
        super(Backbone, self).__init__()
        if model_name == 'resnet50':
            resnet = models.resnet50(pretrained=True)
        self.backbone = nn.Sequential(*list(resnet.children())[:-2])

    def forward(self, x):
        return self.backbone(x)


In [7]:
class FeaturePyramidNetwork(nn.Module):
    def __init__(self, C3_size, C4_size, C5_size, feature_size=256):
        super(FeaturePyramidNetwork, self).__init__()

        self.P5_1 = nn.Conv2d(C5_size, feature_size, kernel_size=1, stride=1, padding=0)
        self.P5_2 = nn.Conv2d(feature_size, feature_size, kernel_size=3, stride=1, padding=1)

        self.P4_1 = nn.Conv2d(C4_size, feature_size, kernel_size=1, stride=1, padding=0)
        self.P4_2 = nn.Conv2d(feature_size, feature_size, kernel_size=3, stride=1, padding=1)

        self.P3_1 = nn.Conv2d(C3_size, feature_size, kernel_size=1, stride=1, padding=0)
        self.P3_2 = nn.Conv2d(feature_size, feature_size, kernel_size=3, stride=1, padding=1)

        self.P6 = nn.Conv2d(C5_size, feature_size, kernel_size=3, stride=2, padding=1)
        self.P7 = nn.Conv2d(feature_size, feature_size, kernel_size=3, stride=2, padding=1)

    def forward(self, C3, C4, C5):
        P5_x = self.P5_1(C5)
        P5_upsampled = F.interpolate(P5_x, scale_factor=2, mode="nearest")
        P5 = self.P5_2(P5_x)

        P4_x = self.P4_1(C4)
        P4_x = P4_x + P5_upsampled
        P4_upsampled = F.interpolate(P4_x, scale_factor=2, mode="nearest")
        P4 = self.P4_2(P4_x)

        P3_x = self.P3_1(C3)
        P3_x = P3_x + P4_upsampled
        P3 = self.P3_2(P3_x)

        P6 = self.P6(C5)
        P7 = self.P7(F.relu(P6))

        return [P3, P4, P5, P6, P7]


In [8]:
class SubNet(nn.Module):
    def __init__(self, num_classes, num_anchors):
        super(SubNet, self).__init__()
        self.conv1 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(256, num_anchors * num_classes, kernel_size=3, padding=1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = self.relu(self.conv4(x))
        return self.conv5(x)

class RetinaNet(nn.Module):
    def __init__(self, num_classes, backbone):
        super(RetinaNet, self).__init__()
        self.backbone = backbone
        fpn_sizes = [256, 512, 1024, 2048]  # Sizes of C3, C4, C5
        self.fpn = FeaturePyramidNetwork(fpn_sizes[0], fpn_sizes[1], fpn_sizes[2])
        self.num_anchors = 9
        self.num_classes = num_classes
        self.classification_head = SubNet(num_classes, self.num_anchors)
        self.regression_head = SubNet(4, self.num_anchors)

    def forward(self, x):
        C3, C4, C5 = self.backbone(x)
        features = self.fpn(C3, C4, C5)
        classifications = [self.classification_head(f) for f in features]
        regressions = [self.regression_head(f) for f in features]
        return classifications, regressions


In [9]:
model = torchvision.models.detection.retinanet_resnet50_fpn(pretrained=True)
num_classes = 91  # COCO has 80 classes + background

model.head.classification_head.num_classes = num_classes

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=RetinaNet_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=RetinaNet_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_coco-eeacb38b.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_coco-eeacb38b.pth
100%|██████████| 130M/130M [00:00<00:00, 147MB/s]  


RetinaNet(
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(inplace=True)
          (downsample): Sequential(
            (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): FrozenBatchNorm2d(256, eps=0.0)


In [10]:
from tqdm import tqdm

def train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10):
    model.train()
    running_loss = 0.0
    data_loader = tqdm(data_loader, desc=f"Epoch {epoch}", unit="batch")
    
    for i, (images, targets) in enumerate(data_loader):
        images = list(image.to(device) for image in images)
        
        valid_targets = []
        for t in targets:
            if len(t) == 0:
                valid_targets.append({'boxes': torch.zeros((0, 4), dtype=torch.float32).to(device),
                                      'labels': torch.zeros((0,), dtype=torch.int64).to(device)})
                continue
            
            boxes = torch.tensor([obj['bbox'] for obj in t], dtype=torch.float32)
            labels = torch.tensor([obj['category_id'] for obj in t], dtype=torch.int64)
            
            if boxes.ndim == 1:
                boxes = boxes.unsqueeze(0)
            
            boxes[:, 2:] += boxes[:, :2]
            
            keep = (boxes[:, 2] > boxes[:, 0]) & (boxes[:, 3] > boxes[:, 1])
            valid_targets.append({
                'boxes': boxes[keep].to(device),
                'labels': labels[keep].to(device)
            })

        loss_dict = model(images, valid_targets)


        classification_loss = loss_dict.get('classification', torch.tensor(0.0, device=device, requires_grad=True))
        bbox_regression_loss = loss_dict.get('bbox_regression', torch.tensor(0.0, device=device, requires_grad=True))


        loss = classification_loss + bbox_regression_loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        data_loader.set_postfix(loss=running_loss / (i + 1))

        if i % print_freq == 0:
            print(f"Epoch: {epoch}, Iteration: {i}, Loss: {loss.item()}")
def evaluate(model, data_loader, device):
    model.eval()
    running_loss = 0.0
    data_loader = tqdm(data_loader, desc="Evaluating", unit="batch")
    
    with torch.no_grad():
        for i, (images, targets) in enumerate(data_loader):
            images = list(image.to(device) for image in images)
            valid_targets = []
            for t in targets:
                if len(t) == 0:
                    valid_targets.append({'boxes': torch.zeros((0, 4), dtype=torch.float32).to(device),
                                          'labels': torch.zeros((0,), dtype=torch.int64).to(device)})
                    continue
                
                boxes = torch.tensor([obj['bbox'] for obj in t], dtype=torch.float32)
                labels = torch.tensor([obj['category_id'] for obj in t], dtype=torch.int64)
                
                if boxes.ndim == 1:
                    boxes = boxes.unsqueeze(0)
                
                boxes[:, 2:] += boxes[:, :2]
                

                keep = (boxes[:, 2] > boxes[:, 0]) & (boxes[:, 3] > boxes[:, 1])
                valid_targets.append({
                    'boxes': boxes[keep].to(device),
                    'labels': labels[keep].to(device)
                })
            
            outputs = model(images)

            running_loss += sum(loss.item() for loss in outputs.values())
            data_loader.set_postfix(loss=running_loss / (i + 1))
            print(outputs)


In [11]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)


In [12]:
num_epochs = 10

for epoch in range(num_epochs):
    train_one_epoch(model, optimizer, train_loader, device, epoch, print_freq=50)
    evaluate(model, val_loader, device)


Epoch 0:   0%|          | 1/29571 [00:03<25:03:38,  3.05s/batch, loss=2.56]

Epoch: 0, Iteration: 0, Loss: 2.5584449768066406


Epoch 0:   0%|          | 51/29571 [00:43<6:09:29,  1.33batch/s, loss=1.78]

Epoch: 0, Iteration: 50, Loss: 1.3251490592956543


Epoch 0:   0%|          | 101/29571 [01:24<6:37:48,  1.23batch/s, loss=1.46]

...

Epoch: 0, Iteration: 7650, Loss: 0.8886857032775879


Epoch 0:  26%|██▌       | 7701/29571 [1:43:48<4:58:54,  1.22batch/s, loss=1.01]

Epoch: 0, Iteration: 7700, Loss: 0.7419942617416382


Epoch 0:  26%|██▌       | 7742/29571 [1:44:41<4:55:12,  1.23batch/s, loss=1.01]


KeyboardInterrupt: 